In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import time

In [2]:
# Load the IMDB dataset
max_features = 5000
maxlen = 100
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)


In [3]:
# Preprocess the data
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [4]:
# Convert sequences back into text
reverse_word_index = dict([(value, key) for (key, value) in imdb.get_word_index().items()])
def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

X_train_text = [decode_review(x) for x in X_train]
X_test_text = [decode_review(x) for x in X_test]

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train_text)

# Preprocess the data
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=maxlen)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=maxlen)

In [5]:
# Create and fit tokenizer
#tokenizer = Tokenizer(num_words=max_features)
#tokenizer.fit_on_texts(X_train)


In [6]:
# Build the model
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
# Train the model
batch_size = 32
epochs = 5
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

In [9]:
start_time = time.time()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)
training_time = time.time() - start_time


Epoch 1/5


782/782 [==============================] - 67s 81ms/step - loss: 0.4254 - accuracy: 0.8023 - val_loss: 0.3970 - val_accuracy: 0.8234
Epoch 2/5
782/782 [==============================] - 73s 93ms/step - loss: 0.2981 - accuracy: 0.8741 - val_loss: 0.3525 - val_accuracy: 0.8440
Epoch 3/5
782/782 [==============================] - 75s 95ms/step - loss: 0.2486 - accuracy: 0.9008 - val_loss: 0.3556 - val_accuracy: 0.8459
Epoch 4/5
782/782 [==============================] - 74s 95ms/step - loss: 0.2109 - accuracy: 0.9149 - val_loss: 0.3968 - val_accuracy: 0.8470
Epoch 5/5
782/782 [==============================] - 80s 102ms/step - loss: 0.1820 - accuracy: 0.9288 - val_loss: 0.4209 - val_accuracy: 0.8401


In [10]:
# Evaluate the model
start_time = time.time()

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
prediction_time = time.time() - start_time

print("Test accuracy:", acc)

782/782 [==============================] - 12s 15ms/step - loss: 0.4209 - accuracy: 0.8401
Test accuracy: 0.8400800228118896


In [11]:
#print("Test accuracy:", acc)
#print("Training time:", training_time, "seconds")
#print("Prediction time:", prediction_time, "seconds")

In [12]:
# Function to preprocess user input
def preprocess_input(user_input):
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)
   
#from tensorflow.keras.preprocessing.text import Tokenizer
    #tokenizer = Tokenizer(num_words=max_features)
    #tokenizer.fit_on_texts(user_input)
    #sequences = tokenizer.texts_to_sequences(user_input)
    #return pad_sequences(sequences, maxlen=maxlen)

In [13]:
# Function to predict sentiment
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "Positive"
    else:
        return "Negative"

In [14]:
# User Interface
print("Welcome to Movie Review Sentiment Analyzer!")

Welcome to Movie Review Sentiment Analyzer!


In [ ]:
while True:
    user_review = input("Enter your movie review (or type 'quit' to exit): ")
    if user_review.lower() == 'quit':
        print("Thank you for using our service!")
        break
    else:
        sentiment = predict_sentiment(user_review)
        print("Predicted sentiment:", sentiment)

Enter your movie review (or type 'quit' to exit): First half has superb dog fight sequences but whose effects are marred by unnecessary songs, irritating characters and a villain who evokes laughter with his Tere Naam look. Hrithik is part time fighter pilot and part time planner of Balakot strikes, who has RAW level info, which he just divulges in front of top officers, probably for great appraisals.
1/1 [==============================] - 0s 332ms/step
Predicted sentiment: Negative
Enter your movie review (or type 'quit' to exit): Yes it's a bit flawed, yes it has few plot holes but, in current crop of movies it's nice change of pace. Like I said it's not a Movie it's an Emotion which simply showcases the fact that if we come to ourselves we can conquer the world.  The action the direction the look and feel of Dog Fights, the gears used by the pilots has realism to it. Yes the love angle may rub few in wrong way but, it's kept very short so I didn't mind it.
1/1 [=====================